In [7]:
import requests
import urllib
import urllib3
from bs4 import BeautifulSoup as bs
from openpyxl import Workbook
import json
import time
import os
import csv
from fake_useragent import UserAgent
from IPython.display import clear_output, display, Image
from IPython.core.display import display, HTML
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [27]:
cn_url="https://i.ktplay.com/app/id101750/topic/21623543"
os_url="https://i.ktplay.com/app/id101780/topic/21628148"
store_dir = "/Users/yuhongc/PycharmProjects/SK_Scraping/0102-New Year Event/元旦活动图片-en/"

In [10]:
driver = webdriver.Chrome()

In [12]:
driver.get(os_url)

In [15]:
#scroll to bottom
start_time = time.time()
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height or time.time()-start_time > 300:
        break
    last_height = new_height

In [28]:
def save_image_from_url(pic_url, savename, save_dir):
    with open(save_dir + savename + ".jpg", 'wb') as handle:
        response = requests.get(pic_url, stream=True)

        if not response.ok:
            return False

        for block in response.iter_content(1024):
            if not block:
                break

            handle.write(block)
        return True

In [17]:
os_source = driver.page_source

In [18]:
# save page source so you don't have to wait another half hour to scroll everything
with open(store_dir+"../page_sources/os_source.txt", "w") as file:
    file.write(driver.page_source)

In [19]:
with open(store_dir+"../page_sources/os_source.txt", "r") as file:
    os_source = file.read()

In [20]:
page_soup = bs(os_source, "html.parser")

In [21]:
replies = page_soup.find_all(class_="reply-item")
replies = replies[::-1]

In [23]:
replies[2].find(class_="content-body")

<div class="content-body"><span class="pre-wrap">#01#+Rogue+C6H806</span></div>

In [24]:
numbers = ["1","2","3","01","02","03"]
hashtags = ["#","＃"]
keys = {"挑战"+str(i):[h+n+h for n in numbers for h in hashtags if str(i) in n] for i in range(1,4)}
keys

{'挑战1': ['#1#', '＃1＃', '#01#', '＃01＃'],
 '挑战2': ['#2#', '＃2＃', '#02#', '＃02＃'],
 '挑战3': ['#3#', '＃3＃', '#03#', '＃03＃']}

In [25]:
wb = Workbook()
ws = wb.active
ws.title = "挑战1"
wb.create_sheet("挑战2")
wb.create_sheet("挑战3")
for sheetname in wb.sheetnames:
    ws = wb[sheetname]
    for col, val in enumerate(["编号","回复内容","昵称"], start=1):
        ws.cell(row=1, column=col).value=val
wb.save(store_dir+"对照信息.xlsx")

In [42]:
numbers = ["1","2","3","01","02","03"]
hashtags = ["#","＃"]
keys = {"挑战"+str(i):[h+n+h for n in numbers for h in hashtags if str(i) in n] for i in range(1,4)}
keywords = sum(keys.values(),[])
submission_counts = {"挑战"+str(i):0 for i in range(1,4)}



def parse_comment(comment):
    nickname = comment.find(class_="").text[:-1]
    comment_text = comment.find(class_="content-body").text
    image_urls = comment.find_all(class_="image")
    
    #check if comment needs to be parsed
    if not image_urls:
        return False
    parse = False
    for keyword in keywords:
        if keyword in comment_text:
            parse = True
            break
    if not parse:
        return False
    
    if comment_text.count("＃")==2 or comment_text.count("#")==2:
        # just one submission
        if len(image_urls)==1:
            # normal people
            for k, v in keys.items():
                for keyword in v:
                    if keyword in comment_text:
                        challenge = k
                        submission_counts[k]+=1
                        if len(comment_text)>40:
                            comment_text = comment_text[:40]
                        if "/" in comment_text:
                            comment_text = comment_text.replace("/","|")
                        text_data = [str(submission_counts[k]), comment_text, nickname]
                        save_image_from_url(image_urls[0]['src']," | ".join(text_data), store_dir+k+"/")
                        ws = wb[k]
                        for col, val in enumerate(text_data, start=1):
                            ws.cell(row=submission_counts[k]+1, column=col).value = val
                        return k
    return comment_text
    

In [45]:
for reply in replies:
    parse_comment(reply)

In [46]:
wb.save(store_dir+"对照信息.xlsx")